In [80]:
from os import getenv
from pymongo import MongoClient
from pandas import DataFrame, Series, concat
from dotenv import load_dotenv
from altair import Chart, X, Y, Color, Tooltip
import requests

In [91]:
filepath = '/Users/jasongersing/PycharmProjects/fastApiProject/.env'
load_dotenv(filepath)
url = getenv('MONGO_URL')
database = "thruTheGrapevine"
collection_GB = "GrapeBuyers"
collection_GS = "GrapeSellers"

In [92]:
db_GB = MongoClient(url)[database][collection_GB]
db_GB.count_documents({})

1000

In [93]:
db_GS = MongoClient(url)[database][collection_GS]
db_GS.count_documents({})

500

In [94]:
grape_buyers = DataFrame(db_GB.find(projection={"_id": False, "grapes_seeking": True, "volume_seeking": True}))
grape_buyers = grape_buyers.explode(column=["grapes_seeking", "volume_seeking"])
grape_buyers.rename(columns={"grapes_seeking": "variety", "volume_seeking": "tons"}, inplace=True)
grape_buyers["side"] = "Buyer"
grape_buyers["tons"] = Series([int(num) for num in grape_buyers["tons"]])
grape_buyers = grape_buyers.groupby(["variety", "side"]).agg({"tons": "sum"}).reset_index()
grape_buyers

,variety,side,tons
0,Cabernet Sauvignon,Buyer,3980
1,Chardonnay,Buyer,4290
2,Granache,Buyer,3810
3,Marsanne,Buyer,3560
4,Merlot,Buyer,4230
5,Mourvedre,Buyer,3875
6,Nebbiolo,Buyer,4020
7,Petit Verdot,Buyer,4275
8,Pinot blanc,Buyer,3595
9,Pinot gris,Buyer,3625


In [95]:
grape_sellers = DataFrame(db_GS.find(projection={"_id": False, "grapes_selling": True, "volume_selling": True}))
grape_sellers = grape_sellers.explode(column=["grapes_selling", "volume_selling"])
grape_sellers.rename(columns={"grapes_selling": "variety", "volume_selling": "tons"}, inplace=True)
grape_sellers["side"] = "Seller"
grape_sellers["tons"] = Series([int(num) for num in grape_sellers["tons"]])
grape_sellers = grape_sellers.groupby(["variety", "side"]).agg({"tons": "sum"}).reset_index()
grape_sellers

,variety,side,tons
0,Cabernet Sauvignon,Seller,1705
1,Chardonnay,Seller,2145
2,Granache,Seller,1670
3,Marsanne,Seller,1510
4,Merlot,Seller,1770
5,Mourvedre,Seller,1100
6,Nebbiolo,Seller,2040
7,Petit Verdot,Seller,1765
8,Pinot blanc,Seller,1895
9,Pinot gris,Seller,2055


In [96]:
total = concat([grape_buyers, grape_sellers])
total

,variety,side,tons
0,Cabernet Sauvignon,Buyer,3980
1,Chardonnay,Buyer,4290
2,Granache,Buyer,3810
3,Marsanne,Buyer,3560
4,Merlot,Buyer,4230
5,Mourvedre,Buyer,3875
6,Nebbiolo,Buyer,4020
7,Petit Verdot,Buyer,4275
8,Pinot blanc,Buyer,3595
9,Pinot gris,Buyer,3625


In [97]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")


def stacked_bar_chart(df: DataFrame, column_1: str, column_2: str, column_3: str) -> Chart:
    return Chart(
        df,
        title=f"{title_fix(column_1)} by {title_fix(column_2)} by {title_fix(column_3)}",
    ).mark_bar().encode(
        x=X(column_3, title=title_fix(column_3), sort="-y"),
        y=Y(column_1),
        color=Color(column_2, title=title_fix(column_2)),
        tooltip=Tooltip([column_2, column_3, column_1])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )

In [98]:
stacked_bar_chart(total, "tons", "side", "variety" )

alt.Chart(...)

In [99]:
chart = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-side').json()
Chart.from_dict(chart)

alt.Chart(...)